In [377]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime
from pyeto import fao

In [378]:
# Abro el archivo
f = open('datos_IDEAM2018', 'r')
lines = f.readlines()
f.close()

'''Abro el catálogo de estaciones para encontrar 
  la información sobre ubicación y altura porque es más 
  fácil y directo que hacerlo con los datos del archivo de texto'''
catalogo = pd.read_csv('/home/diego/Documentos/CATALOGO_ESTACIONES_IDEAM_V10_AGOSTO2017.csv', sep='\t')

In [379]:
# Encuentro una linea de donde estén definidos los meses
for line in lines:
    if 'ENERO *' in line:
        break
        
''' Encuentro los index de las lineas donde está partida la
    columna para los meses'''
index_list = []
index = -1
for i in range(12): # Index para los datos en los 12 meses
    index = line.find('*', index + 1, len(line))
    index_list.append(index)

In [380]:
'''Extraigo en diferentes objetgos dentro de un dict
   las líneas que se tengan con datos para cada estación'''

n = 0 
data_lines = [] # Para almacenar lineas de cada estacion
data_dict = defaultdict() # Para almacenar las lineas de todas las estaciones
est_prec = None # estación anterior
for i, line in enumerate(lines): # Voy por todo el documento
    if 'ESTACION :' in line: # Busco codigo de la estacion
        estacion = int(line.split('ESTACION : ')[-1].split(' ')[0])
        if estacion == est_prec:
            estacion = estacion + 1
    if '************' in line: # Cuento asteriscos de esos del mes
        n += 1
        continue
    if n == 2: 
        data_lines.append(line)
        if 'I D E A M' in line: # Luego del nuevo encabezado inicia una nueva estación
            n = 0
            '''Dependiendo de la longitud de la serie tendrá diferentes cortes'''
            if 'MINIMOS' in data_lines[-2]:
                data_dict[estacion] = data_lines[1:-5] # Almaceno los datos para esa estación
            elif 'MAXIMOS' in data_lines[-2]:
                data_dict[estacion] = data_lines[1:-4] 
            elif 'MEDIOS' in data_lines[-2]:
                data_dict[estacion] = data_lines[1:-3] 
            elif data_lines[-2] == '\n':
                data_dict[estacion] = data_lines[1:-2] 
            else:
                data_dict[estacion] = data_lines[1:-1] 
            data_lines = []
            est_prec = estacion

In [381]:
'''Acá extraigo todos los datos, calculo la radiación solar
   con la ecuación de Angstrom-Prescott y guardo un archivo
   csv con cada estación donde se llamará como el código'''

dias_mes = {1:31, 2:28.5, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

for est in data_dict.keys(): # Iterar sobre las estaciones para extraer la informacion
    if est % 10 == 0:      # Crea lista vacía solo si es el único objeto para esa estación
        tmp_list = [] # Almacena datos
        anno_list = [] # Almacena el año
    
    for j in data_dict[est]: # Extraigo el dato
        for i in index_list:
            try:
                tmp_list.append(float(j[i-7:i]))
            except ValueError: # Si no es un número entonces es un None
                tmp_list.append(None)
            anno_list.append(int(j[:5]))
    if est % 10 != 0:  # Si es la segunda parte, cambia el nombre al normal
        est = est - 1
    
        
    ''' Puede ser que no se encuentre un año dentro del archivo
        entonces en estas lineas garantizaremos al continuidad
        de la serie de tiempo'''
    anno_list_cont = []
    tmp_list_cont = []
    if len(anno_list) <= 0:
        continue
    for i in range(min(anno_list), max(anno_list) + 1):
        for j in range(12):
            anno_list_cont.append(i)
    n = 0
    for i, j in enumerate(anno_list_cont):
        if j in anno_list:
            tmp_list_cont.append(tmp_list[i - n])
        else:
            tmp_list_cont.append(None)
            n += 1
            
    # Para crear la lista de los meses
    n = 0
    mes = 0
    mes_list = []
    while n != len(anno_list_cont):
        mes += 1
        mes_list.append(mes)
        n += 1
        if mes == 12:
            mes = 0             
           
    ### Cálculo de la radiación usando Armstrong Prescott
    try:
        lon = float(catalogo.loc[catalogo['CODIGO CAT.'] == est]['LONGITUD'])
        lon = lon*np.pi/180
        lat = float(catalogo.loc[catalogo['CODIGO CAT.'] == est]['LATITUD'])
        lat = lat*np.pi/180
        alt = int(catalogo.loc[catalogo['CODIGO CAT.'] == est]['ALTITUD'])
    except: # Si no está la estación en el catálogo usa estos valores
        lon = -74*np.pi/180
        lat = 4*np.pi/180
        alt = 0
    
    rad = []
    #Cálculo de la radiación extraterrestre y horas de sol potenciales mensual 
    et_rad_mens = []
    hs_net_mes = []
    for mes in range(1, 13):
        date = datetime(2000, mes, int(dias_mes[mes] // 2)) # Se calcula para mitad del mes
        doy = int(date.strftime('%j')) # dia del año
        sol_dec = fao.sol_dec(doy)
        sha = fao.sunset_hour_angle(lat, sol_dec)  # Ángulo horario a la puesta de sol
        ird = fao.inv_rel_dist_earth_sun(doy)  # Inverso de la distancia relativa tierra - sol
        et_rad = fao.et_rad(lat, sol_dec, sha, ird) # radiación extraterrestre
        et_rad_mens.append(et_rad) 
        hs_net_mes.append(fao.daylight_hours(sha))
    
    # calculo de la rad solar
    sol_rad_list = []
    for mes, hs_mes in zip(mes_list, tmp_list_cont):
        try:
            et_rad = et_rad_mens[mes - 1]
            daylight_hours = hs_net_mes[mes - 1]
            hs_dia = hs_mes / dias_mes[mes] # horas sol media diaria
            sol_rad = fao.sol_rad_from_sun_hours(daylight_hours, hs_dia, et_rad)
            sol_rad_list.append(round(sol_rad * dias_mes[mes], 1))
        except TypeError: # Si no hay valor pues no calcula nada
            sol_rad_list.append(None)

    #Creo el df y lo guardo
    tmp_df = pd.DataFrame({'Year':anno_list_cont, 'Month':mes_list,
                           'Sunshine hours':tmp_list_cont, 'Solar Radiation [MJ/m2]':sol_rad_list})
    tmp_df.to_csv('Brillo/{:d}.csv'.format(est), index=False, sep='\t')